Import all the necessary packages

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from scipy.spatial import distance
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import random
random.seed(42)

import re


Load the files

In [2]:
prompts_train=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
data=pd.merge(summaries, prompts_train, on="prompt_id")


Functions to feature engineer the dataset

In [3]:

def jaccard_similarity(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    intersection = len(words1.intersection(words2))
    union = len(words1) + len(words2) - intersection
    return float(intersection) / float(union)
def cosine_similarity(s1, s2):
    allsentences = [s1 , s2]
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    return round((1-cosine)*100,2)
def word_overlap(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    common_words = words1.intersection(words2)
    overlap_count = len(common_words)
    return overlap_count
def semanticsimilarity(text1, text2):
    score=web_model.predict([(text1,text2)])
    return np.array(score)[0]
def sentiment_diff(text1, text2):
    sentiment_score1 = analyzer.polarity_scores(text1)
    sentiment_score2 = analyzer.polarity_scores(text2)
    return abs(sentiment_score1['compound']-sentiment_score2['compound'])
def is_objective(text):
    subjective_words = ["I", "me", "my", "we", "our", "us", "you", "your", "mine", "ours"]
    
    words = text.split()
    
    for word in words:
        if word.lower() in subjective_words:
            return False  
    
    return True  
# lets put it in the data
data['Jaccard_Similarity'] = data.apply(lambda row: jaccard_similarity(row['prompt_text'], row['text']), axis=1)
data['cosine_similarity'] = data.apply(lambda row: cosine_similarity(row['prompt_text'], row['text']), axis=1)
data['word_overlap'] = data.apply(lambda row: word_overlap(row['prompt_text'], row['text']), axis=1)
#data['sentiment_diff'] = data.apply(lambda row: sentiment_diff(row['prompt_text'], row['text']), axis=1)



below code is obtained from this notebook- https://www.kaggle.com/code/gusthema/commonlit-evaluate-student-summaries-w-tfdf

In [4]:

def count_stopwords(text: str) -> int:
    stopword_list = set(stopwords.words('english'))
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count

# Count the punctuations in the text.
# punctuation_set -> !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count

# Count the digits in the text.
def count_numbers(text: str) -> int:
    numbers = re.findall(r'\d+', text)
    numbers_count = len(numbers)
    return numbers_count

# This function applies all the above preprocessing functions on a text feature.
def feature_engineer(dataframe: pd.DataFrame, feature: str = 'text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].apply(lambda x: len(x.split(' ')))
    dataframe[f'{feature}_length'] = dataframe[feature].apply(lambda x: len(x))
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].apply(lambda x: count_stopwords(x))
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].apply(lambda x: count_punctuation(x))
    dataframe[f'{feature}_number_cnt'] = dataframe[feature].apply(lambda x: count_numbers(x))
    return dataframe

In [5]:
data.head()


,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,Jaccard_Similarity,cosine_similarity,word_overlap
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,0.063636,52.80,21
1,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,0.117347,79.36,46
2,0095993991fe,814d6b,The third wave only started as an experiment w...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,0.090343,62.38,29
3,00c20c6ddd23,814d6b,The experimen was orginally about how even whe...,0.567975,0.969062,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,0.111455,75.27,36
4,00d40ad10dc9,814d6b,The third wave developed so quickly due to the...,-0.910596,-0.081769,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,0.048387,57.09,15


In [6]:
data=feature_engineer(data)

In [7]:

data.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'prompt_question', 'prompt_title', 'prompt_text', 'Jaccard_Similarity',
       'cosine_similarity', 'word_overlap', 'text_word_cnt', 'text_length',
       'text_stopword_cnt', 'text_punct_cnt', 'text_number_cnt'],
      dtype='object')

Separating the data into targer and features

In [8]:
xtrain=data[[ 'Jaccard_Similarity', 'cosine_similarity', 'word_overlap', 'text_word_cnt', 'text_length', 'text_stopword_cnt', 'text_punct_cnt',
       'text_number_cnt']]
y_wording=data['wording']
y_content=data['content']

In [11]:
xtrain.shape

(7165, 8)

In [9]:
y_wording.shape


(7165,)

# split the data into training and test set

In [10]:
X_train_wording, X_test_wording, y_train_wording, y_test_wording = train_test_split(xtrain, y_wording, test_size=0.2, random_state=42)
X_train_content, X_test_content, y_train_content, y_test_content = train_test_split(xtrain, y_content, test_size=0.2, random_state=42)

# Fit a random forest model

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model= RandomForestRegressor()
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model to your data
grid_search.fit(X_train_wording, y_train_wording)

# Get the best hyperparameters
best_params = grid_search.best_params_




model1= RandomForestRegressor()
model1.fit(X_train_wording, y_train_content)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.5s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.4s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   6.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=  10.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.0s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   6.0s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=300; tot

RandomForestRegressor()

In [ ]:
model1.feature_importances_

In [13]:
y_preds = grid_search.predict(X_test_wording)
mse = mean_squared_error(y_test_wording, y_preds)
mse

0.4147938021220113

In [14]:
y_preds = model1.predict(X_test_content)
mse = mean_squared_error(y_test_content, y_preds)
mse

0.2156058408024869

Loading the test data

In [15]:
summaries_test=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
prompts_test=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
test=pd.merge(summaries_test, prompts_test, on='prompt_id')
test['Jaccard_Similarity'] = test.apply(lambda row: jaccard_similarity(row['prompt_text'], row['text']), axis=1)
test['cosine_similarity'] = test.apply(lambda row: cosine_similarity(row['prompt_text'], row['text']), axis=1)
test['word_overlap'] = test.apply(lambda row: word_overlap(row['prompt_text'], row['text']), axis=1)
test=feature_engineer(test)


In [17]:
test.head()

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text,Jaccard_Similarity,cosine_similarity,word_overlap,text_word_cnt,text_length,text_stopword_cnt,text_punct_cnt,text_number_cnt
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...,0.0,50.0,0,3,14,0,0,1
1,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...,0.0,50.0,0,3,14,0,0,1
2,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...,0.0,50.0,0,3,14,0,0,1
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...,0.0,50.0,0,3,14,0,0,1


In [18]:
X_test_data=test[[ 'Jaccard_Similarity', 'cosine_similarity', 'word_overlap', 'text_word_cnt', 'text_length', 'text_stopword_cnt', 'text_punct_cnt',
       'text_number_cnt']]

In [19]:
y_pred = grid_search.predict(X_test_data)
y_pred_content=model1.predict(X_test_data)

In [20]:
submission=pd.DataFrame({'student_id':test.student_id, 'content':y_pred_content, 'wording':y_pred})


In [21]:
submission.to_csv('submission.csv', index=False)


In [22]:
submission

,student_id,content,wording
0,000000ffffff,-1.347873,-1.362276
1,222222cccccc,-1.347873,-1.362276
2,111111eeeeee,-1.347873,-1.362276
3,333333dddddd,-1.347873,-1.362276


[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   8.8s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=  13.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=  13.2s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   8.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=  11.9s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   4.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   4.1s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   8.4s
[CV] END bootstrap=False, max_depth=20, min_samples_le